In [1]:
from PIL import Image
import numpy as np

image_to_read = './example.jpg'
image = Image.open(image_to_read)  # [H, W, rgb]
image = np.asarray(image)

In [2]:
from models.TernausNet import UNet16
from models.MiDas import MidasNet

model = MidasNet()

c:\Users\micha\anaconda3\envs\pytorch-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in C:\Users\micha/.cache\torch\hub\facebookresearch_WSL-Images_main


In [3]:
import torch 
import os

def _read_image(img_rel_path) -> np.ndarray:
    image_to_read = img_rel_path
    image = Image.open(image_to_read)  # [H, W, rgb]
    image = np.asarray(image)
    return image

def _read_rgb_file(rel_path) -> np.ndarray:
    rgb = _read_image(rel_path)
    rgb = np.transpose(rgb, (2, 0, 1)).astype(int)  # [rgb, H, W]
    return rgb

def _load_rgb_data( rgb_rel_path):
    # Read RGB data
    rgb = _read_rgb_file(rgb_rel_path)
    rgb_norm = rgb / 255.0 * 2.0 - 1.0  #  [0, 255] -> [-1, 1]

    outputs = {
        "rgb_int": torch.from_numpy(rgb).float(),
        "rgb_norm": torch.from_numpy(rgb_norm).float(),
    }
    return outputs

img = _load_rgb_data(image_to_read)

In [4]:
img

{'rgb_int': tensor([[[ 41.,  93., 185.,  ..., 187., 169., 153.],
          [ 32.,   0., 122.,  ..., 201., 187., 174.],
          [132.,   8.,  85.,  ..., 198., 187., 180.],
          ...,
          [ 94., 104., 109.,  ..., 160., 172., 176.],
          [116., 127., 134.,  ..., 150., 170., 178.],
          [126., 131., 130.,  ..., 144., 168., 179.]],
 
         [[ 89., 141., 230.,  ..., 177., 161., 149.],
          [ 78.,  47., 167.,  ..., 195., 184., 175.],
          [176.,  53., 130.,  ..., 202., 196., 189.],
          ...,
          [ 76.,  88.,  96.,  ..., 194., 206., 210.],
          [ 95., 109., 118.,  ..., 183., 203., 211.],
          [104., 112., 112.,  ..., 177., 201., 212.]],
 
         [[138., 187., 255.,  ..., 150., 148., 140.],
          [127.,  93., 208.,  ..., 173., 175., 170.],
          [223.,  95., 169.,  ..., 188., 195., 194.],
          ...,
          [ 28.,  37.,  43.,  ...,  12.,  24.,  28.],
          [ 52.,  63.,  69.,  ...,   4.,  24.,  32.],
          [ 63.,  69

In [7]:
ii = img['rgb_int'].unsqueeze(0).to('cuda')

In [8]:
model = model.to('cuda')
pred = model(ii)

c:\Users\micha\anaconda3\envs\pytorch-env\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


OutOfMemoryError: CUDA out of memory. Tried to allocate 184.00 MiB. GPU 

In [ ]:
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torchvision import models


def conv3x3(in_: int, out: int) -> nn.Module:
    return nn.Conv2d(in_, out, 3, padding=1)


class ConvRelu(nn.Module):
    def __init__(self, in_: int, out: int) -> None:
        super().__init__()
        self.conv = conv3x3(in_, out)
        self.activation = nn.ReLU(inplace=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        x = self.activation(x)
        return x


class DecoderBlock(nn.Module):
    def __init__(
        self, in_channels: int, middle_channels: int, out_channels: int
    ) -> None:
        super().__init__()

        self.block = nn.Sequential(
            ConvRelu(in_channels, middle_channels),
            nn.ConvTranspose2d(
                middle_channels,
                out_channels,
                kernel_size=3,
                stride=2,
                padding=1,
                output_padding=1,
            ),
            nn.ReLU(inplace=True),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.block(x)

class Interpolate(nn.Module):
    def __init__(
        self,
        size: int = None,
        scale_factor: int = None,
        mode: str = "nearest",
        align_corners: bool = False,
    ):
        super().__init__()
        self.interp = nn.functional.interpolate
        self.size = size
        self.mode = mode
        self.scale_factor = scale_factor
        self.align_corners = align_corners

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.interp(
            x,
            size=self.size,
            scale_factor=self.scale_factor,
            mode=self.mode,
            align_corners=self.align_corners,
        )
        return x


class DecoderBlockV2(nn.Module):
    def __init__(
        self,
        in_channels: int,
        middle_channels: int,
        out_channels: int,
        is_deconv: bool = True,
    ):
        super().__init__()
        self.in_channels = in_channels

        if is_deconv:
            """
                Paramaters for Deconvolution were chosen to avoid artifacts, following
                link https://distill.pub/2016/deconv-checkerboard/
            """

            self.block = nn.Sequential(
                ConvRelu(in_channels, middle_channels),
                nn.ConvTranspose2d(
                    middle_channels, out_channels, kernel_size=4, stride=2, padding=1
                ),
                nn.ReLU(inplace=True),
            )
        else:
            self.block = nn.Sequential(
                Interpolate(scale_factor=2, mode="bilinear"),
                ConvRelu(in_channels, middle_channels),
                ConvRelu(middle_channels, out_channels),
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.block(x)

num_classes: int = 1,
num_filters: int = 32,
pretrained: bool = False,
is_deconv: bool = False,


pool = nn.MaxPool2d(2, 2)

encoder = torchvision.models.vgg16(pretrained=pretrained).features

relu = nn.ReLU(inplace=True)

conv1 = nn.Sequential(
            encoder[0], relu, encoder[2], relu
)

conv2 = nn.Sequential(
    encoder[5], relu, encoder[7], relu
)

conv3 = nn.Sequential(
    encoder[10],
    relu,
    encoder[12],
    relu,
    encoder[14],
    relu,
)

conv4 = nn.Sequential(
    encoder[17],
    relu,
    encoder[19],
    relu,
    encoder[21],
    relu,
)

conv5 = nn.Sequential(
    encoder[24],
    relu,
    encoder[26],
    relu,
    encoder[28],
    relu,
)

center = DecoderBlockV2(
    512, num_filters * 8 * 2, num_filters * 8, is_deconv
)

dec5 = DecoderBlockV2(
    512 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv
)
dec4 = DecoderBlockV2(
    512 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv
)
dec3 = DecoderBlockV2(
    256 + num_filters * 8, num_filters * 4 * 2, num_filters * 2, is_deconv
)
dec2 = DecoderBlockV2(
    128 + num_filters * 2, num_filters * 2 * 2, num_filters, is_deconv
)
dec1 = ConvRelu(64 + num_filters, num_filters)
final = nn.Conv2d(num_filters, num_classes, kernel_size=1)


c:\Users\micha\anaconda3\envs\pytorch-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\micha\anaconda3\envs\pytorch-env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\micha\anaconda3\envs\pytorch-env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


NameError: name 'DecoderBlockV2' is not defined

In [ ]:

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        conv1 = conv1(x)
        conv2 = conv2(pool(conv1))
        conv3 = conv3(pool(conv2))
        conv4 = conv4(pool(conv3))
        conv5 = conv5(pool(conv4))

        center = center(pool(conv5))

        dec5 = dec5(torch.cat([center, conv5], 1))

        dec4 = dec4(torch.cat([dec5, conv4], 1))
        dec3 = dec3(torch.cat([dec4, conv3], 1))
        dec2 = dec2(torch.cat([dec3, conv2], 1))
        dec1 = dec1(torch.cat([dec2, conv1], 1))
        return final(dec1)